In [1]:
# importing library
import pandas as pd
import numpy as np
import glob
from openpyxl import Workbook
import csv

# read all file in PWD
listing = glob.glob("*.csv")
print(len(listing))

# fixing the extension
new_list = []
for item in listing:
    item = item[0:11] + '.csv'
    new_list.append(item)
print(new_list)

87
['JT.20220401.csv', 'JT.20220402.csv', 'JT.20220403.csv', 'JT.20220404.csv', 'JT.20220405.csv', 'JT.20220406.csv', 'JT.20220407.csv', 'JT.20220408.csv', 'JT.20220409.csv', 'JT.20220410.csv', 'JT.20220412.csv', 'JT.20220413.csv', 'JT.20220414.csv', 'JT.20220415.csv', 'JT.20220416.csv', 'JT.20220417.csv', 'JT.20220418.csv', 'JT.20220419.csv', 'JT.20220420.csv', 'JT.20220421.csv', 'JT.20220422.csv', 'JT.20220423.csv', 'JT.20220424.csv', 'JT.20220425.csv', 'JT.20220426.csv', 'JT.20220427.csv', 'JT.20220428.csv', 'JT.20220429.csv', 'JT.20220501.csv', 'JT.20220502.csv', 'JT.20220503.csv', 'JT.20220504.csv', 'JT.20220505.csv', 'JT.20220506.csv', 'JT.20220507.csv', 'JT.20220508.csv', 'JT.20220509.csv', 'JT.20220510.csv', 'JT.20220511.csv', 'JT.20220512.csv', 'JT.20220513.csv', 'JT.20220514.csv', 'JT.20220515.csv', 'JT.20220516.csv', 'JT.20220517.csv', 'JT.20220518.csv', 'JT.20220519.csv', 'JT.20220521.csv', 'JT.20220522.csv', 'JT.20220523.csv', 'JT.20220524.csv', 'JT.20220525.csv', 'JT.2022

Load The dataset

In [88]:
# read the dataset
df1 = pd.read_csv('JT.20220630.csv', on_bad_lines='skip')

# removing unused field
dropped = ['START.TIME', 'END.TIME', 'THROUGHPUT', 'COMPLETED RECS', 'SELECTION.TIME', 'BULK.NO', 'TXN.MGMT', 'WRITE.CACHE', 'LOCK.COLLISION', 'TOTAL WRITE', 'Unnamed: 15', 'DET PROCESSED', 'DET COMPLETED', 'DET THROUGHPUT', 'DET BULK.NUMBER', 'DET SELECT', 'TYPE OF SELECT', ' DET SELECT TIME']
df1.drop(dropped, axis = 1, inplace = True)

# convert to duration
df1['ELAPSED.TIME'] = pd.to_timedelta(df1['ELAPSED.TIME'])

# USE THIS FOR FIRST ITERATION
# df = df1.copy()
# df.info()

# USE THIS FOR NEXT ITERATION
df = pd.concat([df, df1],ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43494 entries, 0 to 43493
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype          
---  ------          --------------  -----          
 0   BATCH.STAGE     43494 non-null  object         
 1   BATCH           43494 non-null  object         
 2   JOB.NAME        43494 non-null  object         
 3   ELAPSED.TIME    43493 non-null  timedelta64[ns]
 4   PROCESSED RECS  43494 non-null  int64          
dtypes: int64(1), object(3), timedelta64[ns](1)
memory usage: 1.7+ MB


Top 10 job

In [89]:
# Aggregate at job level
job = pd.DataFrame(df.pivot_table(index="JOB.NAME", 
                                    values=["ELAPSED.TIME", "PROCESSED RECS"], 
                                    aggfunc={"ELAPSED.TIME": np.mean, "PROCESSED RECS": np.mean}))
job['PROCESSED RECS'] = job['PROCESSED RECS'].round(0).astype(int)

In [90]:
# sorting
job.sort_values('ELAPSED.TIME', inplace = True, ascending = False)
job.head(10)

,ELAPSED.TIME,PROCESSED RECS
JOB.NAME,,
PRINT.ACCOUNT.STMT,0 days 00:17:08.011494252,4995922
LIMIT.EOD.TIME.BAND,0 days 00:15:07.390804597,3960962
LD.END.OF.DAY.2,0 days 00:11:41.356321839,1182034
EOD.ACCT.ACTIVITY,0 days 00:09:49.482758620,1523842
IDIB.IAK.CALC.GRP.EOM,0 days 00:09:04.500000,20448
IDIB.ATM.SPLIT.FUND,0 days 00:09:01.574712643,803
JLMT.REP.EXPIRED.PRINT,0 days 00:07:13.757225433,1
IDIB.REV.NAB,0 days 00:06:28.344827586,2766211
IC.COB,0 days 00:06:09.364583333,1598503


AVERAGE DURATION

In [91]:
# average duration
Total = job['ELAPSED.TIME'].sum()
print (Total)

0 days 03:41:39.451873550


In [92]:
# remove job name duplicate
clean_df = df.drop_duplicates(['JOB.NAME'],keep=False)

# top 10 table
hasil = job.head(10)

# get top 10 job info
combi_df = hasil.merge(df1, how = 'inner', on = 'JOB.NAME')
combi_df

,JOB.NAME,ELAPSED.TIME_x,PROCESSED RECS_x,BATCH.STAGE,BATCH,ELAPSED.TIME_y,PROCESSED RECS_y
0,PRINT.ACCOUNT.STMT,0 days 00:17:08.011494252,4995922,S920,BNK/AC.STMT.UPDATE,0 days 01:39:38,5218772
1,LIMIT.EOD.TIME.BAND,0 days 00:15:07.390804597,3960962,S020,BNK/SYSTEM.LIMIT.EOD,0 days 00:15:10,4046667
2,LD.END.OF.DAY.2,0 days 00:11:41.356321839,1182034,A100,BNK/LD.END.OF.DAY,0 days 00:16:32,1211362
3,EOD.ACCT.ACTIVITY,0 days 00:09:49.482758620,1523842,S010,BNK/AC.SYS.END.OF.DAY,0 days 00:39:22,8892136
4,IDIB.IAK.CALC.GRP.EOM,0 days 00:09:04.500000,20448,S024,BNK/IDIB.EOM.IAK,0 days 00:08:44,20448
5,IDIB.ATM.SPLIT.FUND,0 days 00:09:01.574712643,803,S015,BNK/IDIB.ATM.SPLIT.FUND,0 days 00:07:43,761
6,JLMT.REP.EXPIRED.PRINT,0 days 00:07:13.757225433,1,D005,BNK/SYSTEM.LIMIT.SOD,0 days 00:05:43,1
7,JLMT.REP.EXPIRED.PRINT,0 days 00:07:13.757225433,1,S020,BNK/SYSTEM.LIMIT.EOD,0 days 00:10:15,1
8,IDIB.REV.NAB,0 days 00:06:28.344827586,2766211,S018,BNK/IDIB.LN.PROV,0 days 00:07:32,2900036
9,IC.COB,0 days 00:06:09.364583333,1598503,D150,BNK/IC.SOD,0 days 00:00:00,0
